### Terminology术语



$o_t$ observation

$a_t$ action

$\pi_{\theta}(a_t|o_t)$ policy 

$\pi_{\theta}(a_t|s_t)$ policy (fully observed) 

state和observation有什么区别？

- observation就是你观察到的，它可能包含推理出状态的完整信息，也可能不;

- state表示所有的信息，也就是fully observed.状态符合马尔可夫性质


### imitation learning

训练数据之间没有独立同分布假设，这与监督学习很大程度上不同，之前的输入会影响后面的状态

### Why behavior cloning fail？

使用监督学习常用的衡量标准不适用


##### 1. 假设条件
$$\pi_\theta(\mathbf{a} \neq \pi^\star(\mathbf{s})|\mathbf{s}) \leq \epsilon \quad \text{for all } \mathbf{s} \in \mathcal{D}_{\text{train}} \text{ 或 } \mathbf{s} \sim p_{\text{train}}(\mathbf{s})$$
含义：在训练数据的状态空间$\mathcal{D}_{\text{train}}$中，策略$\pi_\theta$选择非最优动作的概率不超过$\epsilon$（$\epsilon$是一个小的偏差参数）。


##### 2. 状态分布的分解
$$p_\theta(\mathbf{s}_t) = \underbrace{(1 - \epsilon)^t}_{\text{无错误的概率}} p_{\text{train}}(\mathbf{s}_t) + \underbrace{\left(1 - (1 - \epsilon)^t\right)}_{\text{有错误的概率}} p_{\text{mistake}}(\mathbf{s}_t)$$
- $p_{\text{train}}(\mathbf{s})$：训练数据的状态分布。
- $p_{\text{mistake}}(\mathbf{s})$：因策略选择错误动作而产生的“错误状态分布”。
- 直观理解：策略执行$t$步后，状态$\mathbf{s}_t$的分布由“全程无错误时的训练分布”和“至少一次错误时的其他分布”加权组成。


##### 3. 状态分布的偏差上界
$$|p_\theta(\mathbf{s}_t) - p_{\text{train}}(\mathbf{s}_t)| = \left(1 - (1 - \epsilon)^t\right)|p_{\text{mistake}}(\mathbf{s}_t) - p_{\text{train}}(\mathbf{s}_t)| \leq 2\left(1 - (1 - \epsilon)^t\right)$$
再利用不等式 $(1 - \epsilon)^t \geq 1 - \epsilon t$（对$\epsilon \in [0,1]$成立），进一步推导：
$$\left(1 - (1 - \epsilon)^t\right) \leq \epsilon t \implies |p_\theta(\mathbf{s}_t) - p_{\text{train}}(\mathbf{s}_t)| \leq 2\epsilon t$$
这一步量化了策略偏差$\epsilon$和步数$t$对“策略生成的状态分布与训练分布偏差”的影响——偏差随$\epsilon t$线性增长。


##### 4. 累计成本的上界
$$\sum_t \mathbb{E}_{p_\theta(\mathbf{s}_t)}[c_t] = \sum_t \sum_{\mathbf{s}_t} p_\theta(\mathbf{s}_t) c_t(\mathbf{s}_t) \leq \sum_t \sum_{\mathbf{s}_t} p_{\text{train}}(\mathbf{s}_t) c_t(\mathbf{s}_t) + |p_\theta(\mathbf{s}_t) - p_{\text{train}}(\mathbf{s}_t)| c_{\text{max}}$$
其中$c_t(\mathbf{s})$是状态$\mathbf{s}$在时刻$t$的成本，$c_{\text{max}}$是成本的上界。结合之前的偏差上界，进一步推导：
$$\sum_t \mathbb{E}_{p_\theta(\mathbf{s}_t)}[c_t] \leq \sum_t \epsilon + 2\epsilon t \leq \epsilon T + 2\epsilon T^2$$
这里$T$是总步数。这一结果说明：策略的小偏差$\epsilon$会导致累计成本随$T$（线性项）和$T^2$（二次项）增长，体现了**策略偏差在多步决策中被放大**的特性。


##### 核心结论
这一分析从理论上量化了“策略与最优策略的小偏差”如何通过多步决策传播，导致状态分布偏离训练分布，并最终使累计成本显著增加。它为理解强化学习中策略泛化性的局限提供了数学依据。

## How to address the problem?
- 以更聪明的方式收集数据
  - 故意增加错误及其修正，而不是只收集完美的数据。这样再遇到犯错的情景时机器依然可以学习如何修正
  - 使用数据增强（data augmentation）,比如增加向左向右的摄像机
- 使用更加强大的模型
  - 使用序列模型，比如LSTM，这是因为问题可能是非马尔科夫的
  - Multimodal behavior
    - mixture of guassians
    - latent variable models
    - diffusion models
    - discretization
      - 这张图展示了**自回归离散化（Autoregressive discretization）**的概念，用于处理连续动作空间的离散化建模，常见于强化学习等领域。以下是分部分解释：
      ##### 1. 动作向量表示
      图中左侧的 $\mathbf{a}_t = \begin{pmatrix} 0.1 \\ 1.2 \\ -0.3 \end{pmatrix}$ 是一个连续的动作向量，包含三个维度 $a_{t,0}$、$a_{t,1}$、$a_{t,2}$。自回归离散化的核心是将这一连续向量的每个维度**依次离散化**，并通过概率链式法则建模联合分布。
      ##### 2. 模型结构与离散化过程
      - **卷积编码器（conv net encoder）**：先对输入状态（如图中的驾驶场景图像）进行编码，提取特征。
      - **序列模型模块（sequence model block）**：使用LSTM或Transformer等序列模型，**依次处理每个动作维度**：
        - 第一步：建模 $a_{t,0}$ 的概率分布 $p(a_{t,0}|\mathbf{s}_t)$（对应第一个直方图）。
        - 第二步：基于状态 $\mathbf{s}_t$ 和已确定的 $a_{t,0}$，建模 $a_{t,1}$ 的概率分布 $p(a_{t,1}|\mathbf{s}_t, a_{t,0})$（对应第二个直方图）。
        - 第三步：基于状态 $\mathbf{s}_t$、$a_{t,0}$ 和 $a_{t,1}$，建模 $a_{t,2}$ 的概率分布 $p(a_{t,2}|\mathbf{s}_t, a_{t,0}, a_{t,1})$（对应第三个直方图）。
      ##### 3. 概率链式法则（“Why does this work?”的解释）
      根据概率的链式法则，联合概率 $p(\mathbf{a}_t|\mathbf{s}_t)$ 可以分解为：
      $$
      \begin{align*}
      p(\mathbf{a}_t|\mathbf{s}_t) &= p(a_{t,0}, a_{t,1}, a_{t,2}|\mathbf{s}_t) \\
      &= p(a_{t,0}|\mathbf{s}_t) \cdot p(a_{t,1}|\mathbf{s}_t, a_{t,0}) \cdot p(a_{t,2}|\mathbf{s}_t, a_{t,0}, a_{t,1})
      \end{align*}
      $$
      这一分解让模型可以**逐步离散化每个动作维度**，同时捕捉维度间的依赖关系，从而在离散化后仍能有效建模连续动作的分布。
      这种方法的优势在于，通过自回归的序列建模，将高维连续动作的离散化问题转化为**一维维度的依次离散化**，降低了建模复杂度，同时保留了动作维度间的相关性。

- 使用多任务学习
- DAgger(Dataset Aggregation)
  这张图片是关于**模仿学习（Imitation Learning）**中的**DAgger（Dataset Aggregation）算法**的讲解内容，用于解决让智能体（Agent）的行为策略更频繁有效的问题。以下是详细解释：

##### 核心问题与思路
- 首先提出问题：“Can we make it work more often?” 本质是探讨如何让策略的分布 \( p_{\pi_\theta}(\mathbf{o}_t) \) 尽可能接近真实数据的分布 \( p_{\text{data}}(\mathbf{o}_t) \)。
- 思路是：与其在策略的分布 \( p_{\pi_\theta}(\mathbf{o}_t) \) 上“绞尽脑汁”，不如巧妙地利用真实数据的分布 \( p_{\text{data}}(\mathbf{o}_t) \) 来优化。


##### DAgger算法：Dataset Aggregation
- **目标**：从策略生成的分布 \( p_{\pi_\theta}(\mathbf{o}_t) \) 中收集训练数据，而非仅依赖原始的真实数据分布 \( p_{\text{data}}(\mathbf{o}_t) \)。
- **执行方式**：通过策略 \( \pi_\theta(\mathbf{a}_t|\mathbf{o}_t) \) 来生成数据，但需要为这些数据标注动作 \( \mathbf{a}_t \)。

##### 算法步骤（循环迭代过程）
1. **初始训练**：从人类标注的数据集 \( \mathcal{D} = \{\mathbf{o}_1, \mathbf{a}_1, \dots, \mathbf{o}_N, \mathbf{a}_N\} \) 中训练策略 \( \pi_\theta(\mathbf{a}_t|\mathbf{o}_t) \)。
2. **生成新数据**：运行训练好的策略 \( \pi_\theta(\mathbf{a}_t|\mathbf{o}_t) \)，得到新的观测数据集 \( \mathcal{D}_\pi = \{\mathbf{o}_1, \dots, \mathbf{o}_M\} \)。
3. **人工标注**：请人类为 \( \mathcal{D}_\pi \) 中的观测标注对应的动作 \( \mathbf{a}_t \)。
4. **聚合迭代**：重复上述循环，不断用策略生成新场景的观测、人工标注动作，从而让策略在更丰富的场景下学习，提升“工作频率”和有效性。


##### 背景与意义
DAgger算法由Ross等人于2011年提出，是模仿学习中解决**分布偏移（Distribution Shift）**的关键方法。在传统模仿学习中，若仅用人类演示的“旧数据”训练策略，策略在实际执行时会因场景变化产生分布偏移（即策略生成的场景和人类演示的场景不一致），导致效果下降。DAgger通过“策略生成场景+人类标注动作”的循环，不断扩大训练数据的场景覆盖度，让策略能在更多样的场景下有效工作。
